In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

In [7]:
pipe_data = pd.read_csv('Water_f\pipe_data.csv')
consumption_data = pd.read_csv('Water_f\consumption_data.csv')
reservoir_data = pd.read_csv('Water_f\Reservoir.csv')
station_data = pd.read_csv('Water_f\station_data.csv')
water_quality_data = pd.read_csv('Water_f\water_quality.csv')
city_data = pd.read_csv('Water_f\city_data.csv')  #

In [8]:

X_quality = water_quality_data.iloc[:, :-1].values  
y_quality = water_quality_data.iloc[:, -1].values   


categorical_features = [0] 


preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features)], 
    remainder='passthrough'  
)


pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

X_train, X_test, y_train, y_test = train_test_split(X_quality, y_quality, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

y_pred_quality = pipeline.predict(X_test)

# Print the accuracy
print("Water Quality Prediction Accuracy:", accuracy_score(y_test, y_pred_quality))

Water Quality Prediction Accuracy: 1.0


In [10]:
def get_coordinates(service_point, station_data):
    coords = station_data[station_data['Code'] == service_point][['latitude', 'longitude']]
    if not coords.empty:
        return coords.values[0]  
    else:
        return None

def calculate_distance(pipe_data, station_data):
    distances = []
    valid_coords = []
    for _, pipe in pipe_data.iterrows():
        point_a_coords = get_coordinates(pipe['Service_Point_A'], station_data)
        point_b_coords = get_coordinates(pipe['Service_Point_B'], station_data)
        
        if point_a_coords is not None and point_b_coords is not None:
            # Calculate Euclidean distance
            distance = np.sqrt((point_a_coords[0] - point_b_coords[0]) ** 2 + 
                               (point_a_coords[1] - point_b_coords[1]) ** 2)
            distances.append(distance)
            valid_coords.append((point_a_coords, point_b_coords))  
        else:
            distances.append(np.nan)  
    
    return distances, valid_coords

pipe_data['distance'], valid_coords = calculate_distance(pipe_data, station_data)


X_distance = np.array([coord for pair in valid_coords for coord in pair]) 
y_distance = pipe_data['distance'].dropna().values  


if len(X_distance) != len(y_distance):
    min_len = min(len(X_distance), len(y_distance))
    X_distance = X_distance[:min_len]
    y_distance = y_distance[:min_len]


X_train, X_test, y_train, y_test = train_test_split(X_distance, y_distance, test_size=0.2, random_state=42)

lr_regressor = LinearRegression()
lr_regressor.fit(X_train, y_train)


y_pred_distance = lr_regressor.predict(X_test)


print("Predicted Shortest Distances:", y_pred_distance)

Predicted Shortest Distances: [10.09133914  1.01326258]


In [11]:
def get_coordinates(service_point, station_data):
    coords = station_data[station_data['Code'] == service_point][['latitude', 'longitude']]
    if not coords.empty:
        return coords.values[0] 
    else:
        return None

def calculate_distance(pipe_data, station_data):
    distances = []
    valid_coords = [] 
    for _, pipe in pipe_data.iterrows():
        point_a_coords = get_coordinates(pipe['Service_Point_A'], station_data)
        point_b_coords = get_coordinates(pipe['Service_Point_B'], station_data)
        
        if point_a_coords is not None and point_b_coords is not None:
            # Calculate Euclidean distance
            distance = np.sqrt((point_a_coords[0] - point_b_coords[0]) ** 2 + 
                               (point_a_coords[1] - point_b_coords[1]) ** 2)
            distances.append(distance)
            valid_coords.append((point_a_coords, point_b_coords)) 
        else:
            distances.append(np.nan) 
    
    return distances, valid_coords

pipe_data['distance'], valid_coords = calculate_distance(pipe_data, station_data)

X_distance = np.array([coord for pair in valid_coords for coord in pair])  
y_distance = pipe_data['distance'].dropna().values  

if len(X_distance) != len(y_distance):
    min_len = min(len(X_distance), len(y_distance))
    X_distance = X_distance[:min_len]
    y_distance = y_distance[:min_len]

X_train, X_test, y_train, y_test = train_test_split(X_distance, y_distance, test_size=0.2, random_state=42)

lr_regressor = LinearRegression()
lr_regressor.fit(X_train, y_train)

y_pred_distance = lr_regressor.predict(X_test)

print("Predicted Shortest Distances:", y_pred_distance)

Predicted Shortest Distances: [10.09133914  1.01326258]


In [12]:
X_water_occupied = pipe_data[['pipe_diameter', 'pipe_length']].values
y_water_occupied = pipe_data['water_occupied'].values

X_train, X_test, y_train, y_test = train_test_split(X_water_occupied, y_water_occupied, test_size=0.2, random_state=42)

dt_regressor = DecisionTreeRegressor()
dt_regressor.fit(X_train, y_train)

y_pred_occupied = dt_regressor.predict(X_test)

print("Predicted Water Occupied in Pipe:", y_pred_occupied)
mse = mean_squared_error(y_test, y_pred_occupied)
print("Mean Squared Error:", mse)

Predicted Water Occupied in Pipe: [480. 420.]
Mean Squared Error: 4500.0


In [13]:
X_coverage = station_data[['station_location', 'pipe_length']].values
y_coverage = station_data['covers_all_houses'].values

X_coverage = pd.get_dummies(station_data[['station_location', 'pipe_length']], drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X_coverage, y_coverage, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape[0])
print("Testing set size:", X_test.shape[0])

knn_classifier = KNeighborsClassifier(n_neighbors=1)
knn_classifier.fit(X_train, y_train)

y_pred_coverage = knn_classifier.predict(X_test)

print("Water Supply Coverage Prediction Accuracy:", accuracy_score(y_test, y_pred_coverage))

Training set size: 24
Testing set size: 6
Water Supply Coverage Prediction Accuracy: 0.5


In [14]:
merged_data = pd.merge(city_data, consumption_data, on='city_id')


X_demand = merged_data[['population_density', 'avg_consumption_per_household']].values
y_demand = merged_data['total_consumption'].values

X_train, X_test, y_train, y_test = train_test_split(X_demand, y_demand, test_size=0.2, random_state=42)

demand_regressor = LinearRegression()
demand_regressor.fit(X_train, y_train)

y_pred_demand = demand_regressor.predict(X_test)

print("Predicted Water Demand:", y_pred_demand)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred_demand))

Predicted Water Demand: [2808683.1390994]
Mean Squared Error: 653968419463.6603


In [15]:
X_shortage = consumption_data[['consumption_rate', 'reservoir_capacity']].values
y_shortage = consumption_data['shortage'].values

X_train, X_test, y_train, y_test = train_test_split(X_shortage, y_shortage, test_size=0.2, random_state=42)

svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)

y_pred_shortage = svm_classifier.predict(X_test)

print("Water Shortage Prediction Accuracy:", accuracy_score(y_test, y_pred_shortage))

Water Shortage Prediction Accuracy: 0.0


In [16]:
X_leakage = pipe_data[['pipe_length', 'water_flow_rate']].values


iso_forest = IsolationForest(contamination=0.1)
iso_forest.fit(X_leakage)


leakage_pred = iso_forest.predict(X_leakage)
print("Leakage Detection (1: Normal, -1: Anomaly):", leakage_pred)

Leakage Detection (1: Normal, -1: Anomaly): [ 1  1  1 -1  1  1  1  1  1  1]
